<a href="https://colab.research.google.com/github/StevenVuong/MSc_Project/blob/master/misc4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# ALL THE IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# other imports to handle files
import os
import pickle
import csv

# deep learning imports
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, Convolution1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils, generic_utils, to_categorical
from keras.layers import LeakyReLU
from keras import regularizers

# to split our dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# to mount our drive
from google.colab import drive

In [5]:
# MOUNT THE STORAGE
drive.mount('/content/gdrive')

os.chdir('/content')

# go to where we will be working
print (os.listdir())
os.chdir('gdrive/My Drive/msc_project/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
['.config', 'gdrive', 'sample_data']


**Purpose of this notebook is to create metrics for our final presentation/graphs of everything we might need etc..**

In [0]:
# https://stackoverflow.com/questions/50920908/get-confusion-matrix-from-a-keras-multiclass-model
# Build confusion matrices for train and test data
# 3d is in Step3, need to run metrics on 4th batch of processed_brains_aug
# 2d is in misc3, need to run prediction metrics on 4th batch of processed_brains_aug also
# age/sex is in misc2 and runs off the larger dataset, pulling from the Control CSV.

Best models so far:
-  74.138% on the 3D brainscans, model 80
-  100% on the 2D brainscans(slice86), model keep100
-  76.667% on the Age/Sex values, each have a deep learning model
  

Metrics to present:
 <br>
-  Final accuracy and losses
-  Graphs of training accuracy and losses
-  Confusion Matrices
-  Deep learning architecture diagrams


Information to gather:
<br>
-  Quality of each of the samples, where they came from. Distribution, sizes etc..

In [0]:
## Architecture Diagrams http://alexlenail.me/NN-SVG/AlexNet.html

 Attempt at confusion matrix here**
 <br>
Logic:
-  Load model, load the data for both train and test
-  Get true values, get predicted values for both train and test
-  Convert back to binary
-  Input into confusion matrix

In [0]:
def to_binary(cat_array):
  '''Function to convert categorical back to binary values'''
  binary_output_array = []
  for i in range(len(cat_array)):
    binary_output_array.append(np.argmax(cat_array[i]))
    
  binary_output_array = np.array(binary_output_array)
  return binary_output_array

In [0]:
def get_86_slice(ts):
  '''Gets the 86th slice and return this'''
  returned_array = []
  for ax_slice in total_slices:
    returned_array.append(ax_slice[86,:,:,0])
    
  returned_array = np.array(returned_array)
  return returned_array

In [0]:
# gets our y-values and converts to keras, one hot encoded outputs
def get_y_values(total_slices_info):
  
  y_values = [s[2] for s in total_slices_info]

  y_values = np.array(to_categorical(y_values, 2))
  
  return y_values

**BELOW IS FOR OUR 2D BRAINSCANS**

In [0]:
## Load our model, start with 100% accuracy model on the 2d scans
model_2dscans = pickle.load( open( "all_mprage_grappa/z_tests/keep_models/100Model(2dscans).pkl", "rb" ) )

# load training data for this model

      y_values = get_y_values(total_slices_info)
      slices_86 = get_86_slice(total_slices) # get 86th slice
      slices_86 = np.expand_dims(slices_86, axis=3)

In [18]:
## Loop through all our total slices and accumulate all the total slices info
training_batch_f = os.listdir('all_mprage_grappa/processed_brains_aug/')[:3]
print (training_batch_f)

# initialise our set of predicted outputs
y_preds = [] 
# Just loop through once because we want to predict using our model only
for tbf in training_batch_f:
  with open('all_mprage_grappa/processed_brains_aug/'+tbf, 'rb') as f:
    total_slices, total_slices_info = pickle.load(f) 

    # get y values and process each slice
    y_values = get_y_values(total_slices_info)
    slices_86 = get_86_slice(total_slices) # get 86th slice
    slices_86 = np.expand_dims(slices_86, axis=3)

    # Evaluate our model against this 
    train_predict = model_2dscans.evaluate(x=slices_86, y=y_values, verbose=1)
    y_preds.append(train_predict)
    
# set of batches that we will tpredict
y_preds = np.array(y_preds)

['dbatch0.pkl', 'dbatch1.pkl', 'dbatch2.pkl']
100/100 [==============================] - 1s 14ms/step


In [17]:
np.shape(y_preds[0])

()

In [0]:
## Then test with /dbatch3.pkl

**BELOW IS FOR OUR 3D BRAINSCANS**

**BELOW IS FOR OUR AGE/SEX MODEL**